<a href="https://colab.research.google.com/github/GuidoGiacomoMussini/DeepLearning-automated_diagnosis_of_pigmented_skin_lesions/blob/main/Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Skin Lesion Project - Notebook 1**

---

##7 Classes Classification:

* akiec      
* bcc     
* bkl 
* df    
* mel   
* nv 
* vasc     
     

---
Guido Giacomo Mussini 988273

---













# **0 - Initialization**

## 0.1 - Libraries 

In [ ]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 8.2 MB/s eta 0:00:00


In [ ]:
import os
import shutil
from glob import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#%matplotlib inline

from PIL import Image

import random as rnd
from IPython.display import clear_output 
from tqdm.notebook import tqdm_notebook
from collections import defaultdict, Counter, OrderedDict
from imblearn.over_sampling import SMOTE

import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchmetrics.classification import MulticlassAccuracy, BinaryAccuracy
from torchmetrics.classification import MulticlassConfusionMatrix
from torch import nn
from torch import optim
from torchvision import models
from torchsummary import summary
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchtext.vocab import vocab

import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools

## 0.2 - Functions

Model prediction: 

* train the model and test the performance.
* Input: train and test set, number of epochs, the loss function, the optimizer and a boolean value which shows the progression of the algorithm in the epochs
* It returns 2 dictionaries containing information about Accuracy and Loss of training and test set
* inspired by: https://github.com/dash-ka/DL_Natural_Language_Processing




In [ ]:
def Prediction(model, train_loader, test_loader, n_epochs, criterion, optimizer, show_progress): 

  model_name = model.__class__.__name__
  loss_dict, acc_dict = defaultdict(list), defaultdict(list)
  tr_acc, tr_tot, train_accuracy, val_acc, val_tot, validation_accuracy = 0,0,0,0,0,0

  for epoch in tqdm_notebook(range(n_epochs)):

    rtrain_loss = 0
    model.train()
    #train
    for batch_index, tr_batch in enumerate(train_loader, start = 1):

        #retrieve  the data
        images, labels = tr_batch
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        #forword step
        y_pred = model(images)
        #loss 
        loss = criterion(y_pred, labels)
        #backprop
        loss.backward()
        #param updates
        optimizer.step() 
        # clear the gradient 
        model.zero_grad()

        # save results
        #train_loss.append(loss.item())
        batch_loss = loss.item()
        rtrain_loss += (batch_loss - rtrain_loss) / batch_index  
        #accuracy
        _, predicted = torch.max(y_pred, 1)
        tr_tot += labels.size(0)
        tr_acc += (predicted == labels).sum().item()


    train_accuracy = (tr_acc)/tr_tot
    acc_dict["training_accuracy"].append(train_accuracy)
    loss_dict["training_loss"].append(rtrain_loss)

    #update the gradient modifier
    scheduler.step(rtrain_loss)

    model.eval()
    rval_loss = 0
    #eval
    for batch_index, val_batch in enumerate(test_loader, start = 1):

      #get the data
      images, labels = val_batch
      images = Variable(images).to(device)
      labels = Variable(labels).to(device)

      with torch.no_grad():
        #prediction
        val_pred = model(images)
        #prediction loss
        v_loss = criterion(val_pred, labels)

        #save results
        #val_loss.append(loss.item())
        batch_loss = v_loss.item()
        rval_loss += (batch_loss - rval_loss) / batch_index 
        #accuracy
        _, predicted = torch.max(val_pred, 1)
        val_tot += labels.size(0)
        val_acc += (predicted == labels).sum().item()

    validation_accuracy = (val_acc)/val_tot
    acc_dict["validation_accuracy"].append(validation_accuracy)
    loss_dict["validation_loss"].append(rval_loss)
    
    if show_progress == True:
      print('[Model: %s ] -> [epoch %d]: \n [train Loss %.5f], [val Loss %.5f] \n [train Acc  %.5f], [val Acc  %.5f]' \
            % (model_name, epoch, rtrain_loss, rval_loss, train_accuracy, validation_accuracy ))
      print("---------------------------")

  return loss_dict, acc_dict

Test the model on the train set

* inspired by: https://www.kaggle.com/code/unstructuredrahul/deep-learning-pytorch-binary-classification?scriptVersionId=67067211&cellId=38

In [ ]:
def test_pred(model, test_loader):
  with torch.no_grad():

    preds = []
    model.eval()
    for i, data in enumerate(test_loader):

      images, labels = data
      images = Variable(images)
      labels = Variable(labels)
      labels = labels.float()

      y_test_pred = model(images)
      y_test_pred = y_test_pred.argmax(axis=1)
      preds.append(y_test_pred.detach().numpy())

  return preds

Function used to fix the indexes of the dataset

In [ ]:
def re_index(data):
  s = pd.Series(range(len(data)))
  data = data.set_index(s)
  return data

Find the accuracy of the test prediction

In [ ]:
def Accuracy(Dataframe):
  Mat = Dataframe.to_numpy()
  TP = 0
  Tot = 0
  for i in range(0,7):
    for j in range(0,7):
      Tot = Mat[i][j] + Tot
      if i == j: TP = Mat[i][j] + TP
  return TP/Tot

## 0.3 - Import the Data From Kaggle (estimated time: 4 m)

In [ ]:
!mkdir WD

! #mkdir ~/.kaggle
! #cp kaggle.json ~/.kaggle/
! #chmod 600 ~/.kaggle/kaggle.json
os.environ['KAGGLE_USERNAME'] = "guidomussini"
os.environ['KAGGLE_KEY'] = "f7b24d630bc3e7e7fda7a5a1b32f4582"
! kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
! unzip /content/skin-cancer-mnist-ham10000.zip -d /content/WD

#remove useless data 
shutil.rmtree('/content/WD/ham10000_images_part_1')
shutil.rmtree('/content/WD/ham10000_images_part_2')
! rm '/content/WD/hmnist_28_28_L.csv'
! rm '/content/WD/hmnist_28_28_RGB.csv'
! rm '/content/WD/hmnist_8_8_L.csv'
! rm '/content/WD/hmnist_8_8_RGB.csv'
! rm '/content/skin-cancer-mnist-ham10000.zip'

# **1 - Define the dataset**




##1.1 - Merge the images from the 2 folders

*   Code taken by: https://www.kaggle.com/code/sid321axn/step-wise-approach-cnn-model-77-0344-accuracy


Merge the images of the 2 folders

In [ ]:
base_skin_dir = os.path.join('..', 'content/WD')
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

sdf = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))

create a column in which each row contain the path to a image

In [ ]:
sdf = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))
sdf['path'] = sdf['image_id'].map(imageid_path_dict.get)

## 1.2 - Metadata Handling

Check for duplicate images

In [ ]:
sdf = sdf.drop_duplicates(subset=['lesion_id']) 
sdf = sdf.drop_duplicates(subset=['image_id']) 

print(sdf.shape[0]) #now i have 7470 lesions 

7470


Check for Missing Values

In [ ]:
print("number of non-NA data per columns:\n",sdf.isnull().sum())
#some NA in age -> since that column will be removed from the dataset, i don't impute them

number of non-NA data per columns:
 lesion_id        0
image_id         0
dx               0
dx_type          0
age             52
sex              0
localization     0
path             0
dtype: int64


# **2 - Data Visualisation**

Lesion distribution: 
* **green** if the lesion is **benign**
* **red** if the lesion is **malignant**

In [ ]:
class_perc = round((sdf["dx"].value_counts() / len(sdf["dx"])), 2)
#nv       0.72
#bkl      0.10
#mel      0.08
#bcc      0.04
#akiec    0.03
#vasc     0.01
#df       0.01
plt.bar(class_perc.index, class_perc, color = ["green", "green", "red", "red", "red", "green", "green"])

# **3 - Prepare the Data**

## 3.1 - Generate the Training (train + val) and Test set

Encode the response variable

In [ ]:
sdf = re_index(sdf)
sdf['label'] = pd.Categorical(sdf['dx']).codes
sdf = sdf[['path', 'label']]

#this function, defined in the section 'functions', fix the row-index of the dataset
sdf = re_index(sdf)

sample the training and the test set

In [ ]:
train_val, test = train_test_split(sdf, test_size=0.30,random_state=19)
#need to convert x_train_val in an array to merge it in the future with the augumented data
x_train_val = train_val[['path']].to_numpy() 

#re index the df 
train_val = re_index(train_val)
test = re_index(test)

print("Training set:", len(train_val), "\nTest set:", len(test))

Training set: 5229 
Test set: 2241


## 3.2 - Store the images as arrays (Expected time: 3m)
 
 The images are sized as 32x32 since speed up the computation of the algorithm mantaining a good amount of information

In [ ]:
sh_x = 32
sh_y = 32
train_val['image'] = train_val['path'].map(lambda x: np.asarray(Image.open(x).resize((sh_x,sh_y))))
test['image'] = test['path'].map(lambda x: np.asarray(Image.open(x).resize((sh_x,sh_y))))

#delete useless variables
train_val = train_val[['image', 'label']]
test = test[['image', 'label']]

#trasform in array for the next steps
x_train_val = np.asarray(train_val['image'].tolist())
y_train = np.array(train_val['label'])


## 3.3 - Data Augumentation

Since the training set contain around 5000 observations, a good practice is to generate synthetic images in order to increase the number of observation. 

I have a random mix of vertical and horizontal flip to augument the data. In that way the dataset cardinality is doubled

**Note that** the augumented data have been used only in the training set (train +validation), while in the test set has been used only original data.

In [ ]:
aug = list()
#T.RandomPerspective(distortion_scale=0.4, p=1.0), \
transforms = T.RandomApply(torch.nn.ModuleList([T.RandomVerticalFlip(p=1.0), \
                                                T.RandomHorizontalFlip(p=1.0)]), p=1)

#augumenter = T.RandomPerspective(distortion_scale=0.6, p=1.0)
aug = train_val["image"].map(lambda x: transforms(torch.tensor(x).permute(2,0,1)))

for i in range(len(aug)):
  aug[i] = np.array(aug[i])

Create the dataset containing the original data and the augumented ones

In [ ]:
#trasform the augumented data as an array
aug = np.asarray(aug.tolist())
aug = aug.reshape(aug.shape[0], *(sh_x, sh_y, 3))

#trasform the images into array
x_train_val = np.asarray(x_train_val.tolist())
x_train_val = x_train_val.reshape(x_train_val.shape[0], *(sh_x, sh_y, 3))

In [ ]:
#concatenete the 2 arrays
df_images = np.concatenate((aug, x_train_val))

#double the labels to make them fit with the original data and the concatenated augumented data
df_labels = np.concatenate((y_train, y_train))
#df_labels = np.concatenate((y_train_Smote, y_train))

#define the dataset containing training and val examples
coln1 = {'image': list(df_images), 'label': list(df_labels)}
trainval = pd.DataFrame(data=coln1)

print("len training:", len(df_labels))

len training: 10458


## 3.4 - Split the training set in Train and Validation set

In [ ]:
train, val = train_test_split(trainval, test_size=0.30,random_state=19)
#fix the indexes
train = re_index(train)
val = re_index(val)
print("train:", len(train), "\nval:", len(val))

train: 7320 
val: 3138


## 3.5 - Define features and response variable for train, validation and test set

In [ ]:
#train
x_train = train.drop(["label"], axis = 1)
x_train = np.asarray(x_train['image'].tolist())
y_train = train["label"]
y_train = np.array(y_train)

#validation
x_val = val.drop(["label"], axis = 1)
x_val = np.asarray(x_val['image'].tolist())
y_val = val["label"]
y_val = np.array(y_val)

#test
x_test = test.drop(["label"], axis = 1)
x_test = np.asarray(x_test['image'].tolist())
y_test = test["label"]
y_test = np.array(y_test)

## 3.6 - Reshape and convert the images in the correct format

all the images have been shaped as 32x32x3 images

In [ ]:
#reshape to be sure that they are in the right shape
x_train = x_train.reshape(x_train.shape[0], *(sh_x, sh_y, 3))
x_val = x_val.reshape(x_val.shape[0], *(sh_x, sh_y, 3))
x_test = x_test.reshape(x_test.shape[0], *(sh_x, sh_y, 3))

In [ ]:
# converting the images in torch format
x_train  =torch.from_numpy(x_train.astype(np.float32))
x_val  = torch.from_numpy(x_val.astype(np.float32))
x_test  = torch.from_numpy(x_test.astype(np.float32))

#converting the lables in torch format
y_train = torch.from_numpy(y_train.astype(np.float32))
y_val = torch.from_numpy(y_val.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


## 3.7 - Normalization 

All the images are normalized

In [ ]:
x_train = TF.normalize(x_train, x_train.mean(), x_train.std())
x_val = TF.normalize(x_val, x_val.mean(), x_val.std())
x_test = TF.normalize(x_test, x_test.mean(), x_test.std())

# **4 - CNN Framework**


## 4.1 Pytorch Dataset and Dataloader

use GPU if possible

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define the Class Data

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = X.permute(0,3,1,2)
        self.y = y.type(torch.LongTensor)
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

Define train, validation and test loader

In [ ]:
batch_size = 64 

train_set = Data(x_train, y_train)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

val_set = Data(x_val, y_val)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True)

test_set = Data(x_test, y_test)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

## 4.2- Model1

In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.ReLU = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=(5, 5))
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.drop40 = nn.Dropout(0.2)
        self.drop20 = nn.Dropout(0.5)
        self.LSM= torch.nn.LogSoftmax(dim=1)
        

        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding = "same")
        self.norm2d1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), padding = "same")
        self.norm2d2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3,3), padding = "same")
        self.norm2d3 = nn.BatchNorm2d(128)
 
        self.flat = nn.Flatten()

        self.linear1 = nn.Linear(128, 64)
        self.norm1d1 = nn.BatchNorm1d(64)
        self.linear2 = nn.Linear(64, 42)
        self.norm1d2 = nn.BatchNorm1d(42)
        self.linear3 = nn.Linear(42, 7)

 
    def forward(self, x):
  
        x = self.ReLU(self.conv1(x))
        x = self.pool5(x)
        x = self.norm2d1(x)
        x = self.drop40(x)

        x = self.ReLU(self.conv2(x))
        x = self.pool3(x)
        x = self.norm2d2(x)
        x = self.drop40(x)

        x = self.ReLU(self.conv3(x))
        x = self.pool2(x)
        x = self.norm2d3(x)
        x = self.drop40(x)

        x = self.flat(x)

        x = self.ReLU(self.linear1(x))
        x = self.norm1d1(x)
        x = self.drop20(x)

        x = self.ReLU(self.linear2(x))
        x = self.norm1d2(x)
        x = self.drop20(x)

        x = self.LSM(self.linear3(x))

        return x

## 4.3 - ModelN

* Based on: https://www.nature.com/articles/s41598-022-22644-9#Tab2

In [ ]:
class ModelN(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.ReLU = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3))
        self.drop25 = nn.Dropout(0.25)
        self.drop50 = nn.Dropout(0.50)
        self.softmax = torch.nn.Softmax(dim=1)
        self.sigmoid = torch.nn.Sigmoid()
        self.LSM= torch.nn.LogSoftmax(dim=1)
        

        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding = "same")
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), padding = "same")
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3,3), padding = "same")
        self.norm3 = nn.BatchNorm2d(128)
 
        self.flat = nn.Flatten()

        self.linear1 = nn.Linear(576, 1024)
        self.normL1 = nn.BatchNorm1d(1024)
        self.linear2 = nn.Linear(1024, 256)
        self.normL2 = nn.BatchNorm1d(256)
        self.linear3 = nn.Linear(256, 42)
        self.normL3 = nn.BatchNorm1d(42)
        self.linear4= nn.Linear(42, 7)
 
    def forward(self, x):
  
        x = self.ReLU(self.conv1(x))
        x = self.pool2(x)
        x = self.norm1(x)
        x = self.drop25(x)

        x = self.ReLU(self.conv2(x))
        x = self.pool2(x)
        x = self.norm2(x)
        x = self.drop25(x)

        x = self.flat(x)

        x = self.ReLU(self.linear1(x))
        x = self.normL1(x)
        x = self.drop50(x)

        x = self.ReLU(self.linear2(x))
        x = self.normL2(x)
        x = self.drop50(x)

        x = self.ReLU(self.linear3(x))
        x = self.normL3(x)
        x = self.drop50(x)

        x = self.LSM(self.linear4(x))

        return x

## 4.4 - Model Training

ModelN

In [ ]:
n_epochs = 100
model = ModelN() #Model1
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    factor=0.1, 
    patience=10, 
    verbose=True)
criterion = nn.CrossEntropyLoss().to(device)

lossN, accN = Prediction(model, train_loader, val_loader, n_epochs, criterion, optimizer, show_progress = False)

Model1 

In [ ]:
n_epochs = 50
model = Model1()
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    factor=0.1, 
    patience=10, 
    verbose=True)
criterion = nn.CrossEntropyLoss().to(device)

loss1, acc1 = Prediction(model, train_loader, val_loader, n_epochs, criterion, optimizer, show_progress = True)

print the summary of the model

In [ ]:
#summary(model, (3,32,32))

## 4.5 Result visualization

Accuracy

In [ ]:
plt.plot(acc1["training_accuracy"], label = "training accuracy") #acc1
plt.plot(acc1["validation_accuracy"], label = "validation accuracy") #acc1
plt.legend()
plt.title("Accuracy")
plt.show()

Loss

In [ ]:
plt.plot(loss1["training_loss"], label = "training loss") #loss1
plt.plot(loss1["validation_loss"], label = "validation loss") #loss1
plt.title("Loss")
plt.title("Loss")
plt.legend()
plt.show()

## 4.6 - Prediction on the Test set

In [ ]:
preds = test_pred(model, test_loader)

trasform the data to build the confusion matrix

In [ ]:
y_pred_list = [a.squeeze().tolist() for a in preds]

#last batch has only one element --> i temporary remove it to apply vstack.
y_pred_list1 = y_pred_list[:-1]
last_element = y_pred_list[-1]

p = np.vstack(y_pred_list1)
ytest_pred = list(itertools.chain.from_iterable(p))

#append the last element
ytest_pred.insert(len(ytest_pred), last_element)


y_true_test = y_test.ravel()

Confusion Matrix

In [ ]:
a = conf_matrix = confusion_matrix(y_true_test, ytest_pred)
axis = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
adf = pd.DataFrame(a, columns = axis)
adf.index = axis
sns.heatmap(adf, annot=True, fmt = 'g', cmap="Blues")

Accuracy

In [ ]:
Accuracy(adf)